# OpenAI Agents SDK - 메모리 관리

이 튜토리얼은 OpenAI Agents SDK에서 에이전트의 메모리를 관리하는 다양한 방법을 다룬다. 기본적인 무상태 동작부터 수동 메모리 관리, 컨텍스트 기반 메모리, 그리고 복합 메모리 패턴까지 단계별로 학습한다.

In [ ]:
# 필요한 패키지 설치
!pip install openai-agents python-dotenv

---

## 1. 무상태 에이전트 - 기본 동작 이해

이 챕터에서는 에이전트의 기본 동작인 무상태(stateless) 특성을 이해한다. 기본적으로 각 `Runner.run_sync()` 호출은 독립적이며, 에이전트는 이전 대화를 기억하지 않는다. 이 특성을 이해해야 메모리 관리의 필요성과 구현 방법을 올바르게 파악할 수 있다.

### 학습 내용

- 각 `Runner.run_sync()` 호출이 독립적으로 실행되는 방식
- 에이전트가 기본적으로 이전 대화를 기억하지 않는 이유
- 명시적 메모리 관리가 필요한 상황 이해
- 메모리 기능 추가 전의 기준 동작 파악

In [1]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner

# 환경 변수 로드
load_dotenv()

MODEL = "gpt-4o-mini"

### 무상태 동작 시연

기본 에이전트를 생성하고 두 번의 독립적인 호출을 수행한다. 첫 번째 호출에서 사용자가 자신의 이름을 알려주고, 두 번째 호출에서 이름을 물어본다. 에이전트는 각 호출을 완전히 독립적으로 처리하기 때문에 첫 번째 대화의 내용을 기억하지 못한다. 이것이 에이전트의 기본 동작이다.

In [2]:
# 기본 에이전트 생성
agent = Agent(
    name="Conversationalist", 
    instructions="당신은 도움이 되는 어시스턴트입니다."
)

# 첫 번째 상호작용
print("=== 첫 번째 상호작용 ===")
result1 = await Runner.run(agent, "제 이름은 Alice입니다.")
print(f"사용자: 제 이름은 Alice입니다.")
print(f"에이전트: {result1.final_output}")

=== 첫 번째 상호작용 ===
사용자: 제 이름은 Alice입니다.
에이전트: 안녕하세요, Alice님! 만나서 반갑습니다. 어떻게 도와드릴까요?


In [3]:
# 두 번째 상호작용 - 에이전트는 이름을 기억하지 못한다
print("\n=== 두 번째 상호작용 ===")
result2 = await Runner.run(agent, "제 이름이 뭐죠?")
print(f"사용자: 제 이름이 뭐죠?")
print(f"에이전트: {result2.final_output}")

print("\n[참고] 에이전트는 이전 대화를 기억하지 못하므로 이름을 알 수 없다고 응답한다.")


=== 두 번째 상호작용 ===
사용자: 제 이름이 뭐죠?
에이전트: 현재까지 대화에서 사용자의 이름이 언급되지 않았어요. 이름을 알려주시면 앞으로 대화할 때 사용해드릴 수 있습니다!

[참고] 에이전트는 이전 대화를 기억하지 못하므로 이름을 알 수 없다고 응답한다.


---

## 2. 영속 메모리 - 자동 대화 기록

이 챕터에서는 `run_demo_loop`를 사용하여 자동으로 대화 기록을 유지하는 방법을 다룬다. 이 함수는 여러 상호작용에 걸쳐 대화 컨텍스트를 자동으로 관리해주므로, 멀티턴 대화를 쉽게 구현할 수 있다.

### 학습 내용

- `run_demo_loop`를 사용한 영속 대화 세션
- 여러 상호작용에 걸친 자동 메모리 관리
- 대화 전반에 걸쳐 컨텍스트를 기억하는 에이전트 구축
- 원활한 멀티턴 대화 기능

### 주의사항

`run_demo_loop`은 터미널 기반의 대화형 세션을 위해 설계되었으므로, Jupyter 노트북 환경에서는 정상적으로 동작하지 않을 수 있다. 이 코드는 Python 스크립트(`.py` 파일)로 실행하는 것을 권장한다.

In [4]:
import os
import asyncio
from dotenv import load_dotenv
from agents import Agent, run_demo_loop

### 영속 메모리 에이전트 (터미널 전용)

`run_demo_loop` 함수는 대화 기록을 자동으로 유지하면서 연속적인 대화 세션을 제공한다. 사용자가 'quit'을 입력할 때까지 대화가 계속되며, 에이전트는 이전 대화 내용을 기억한다. 아래 코드는 `.py` 파일로 저장하여 터미널에서 실행해야 한다.

```python
# 이 코드를 persistent_memory.py로 저장하여 실행하라
# python persistent_memory.py

import os
import asyncio
from dotenv import load_dotenv
from agents import Agent, run_demo_loop

load_dotenv()

async def main():
    agent = Agent(
        name="FriendlyAssistant",
        instructions="당신은 기억력이 좋은 도움이 되는 어시스턴트입니다."
    )
   
    print("대화를 시작한다. 종료하려면 'quit'을 입력하라.")
    await run_demo_loop(agent)

if __name__ == "__main__":
    asyncio.run(main())
```

In [5]:
# 영속 메모리 에이전트 정의
persistent_agent = Agent(
    name="FriendlyAssistant",
    instructions="당신은 기억력이 좋은 도움이 되는 어시스턴트입니다."
)

print(f"영속 메모리 에이전트 '{persistent_agent.name}'가 생성되었다.")
print("이 에이전트는 run_demo_loop와 함께 사용하면 대화 기록을 자동으로 유지한다.")

영속 메모리 에이전트 'FriendlyAssistant'가 생성되었다.
이 에이전트는 run_demo_loop와 함께 사용하면 대화 기록을 자동으로 유지한다.


---

## 3. 수동 메모리 관리

이 챕터에서는 `result.to_input_list()`를 사용하여 대화 기록을 수동으로 관리하는 방법을 다룬다. 이 패턴은 대화 컨텍스트에 대한 세밀한 제어가 필요할 때 유용하다.

### 학습 내용

- `result.to_input_list()`를 사용한 대화 기록 추출
- 후속 호출을 위한 대화 컨텍스트 수동 구축
- 기억할 내용에 대한 세밀한 제어
- 커스텀 메모리 관리 패턴 구축

### to_input_list() 메서드 이해

`RunResult` 객체의 `to_input_list()` 메서드는 현재까지의 대화 기록을 LLM에 전달할 수 있는 형식의 리스트로 변환한다. 이 리스트에 새로운 사용자 메시지를 추가하고 다음 호출에 전달하면, 에이전트는 이전 대화의 맥락을 유지한 채 응답할 수 있다. 이 방식은 어떤 대화 내용을 기억하고 어떤 내용을 제외할지 직접 제어할 수 있다는 장점이 있다.

In [6]:
import os
import asyncio
from dotenv import load_dotenv
from agents import Agent, Runner

In [7]:
async def manual_memory_demo():
    """수동 메모리 관리를 시연하는 함수이다."""
    
    agent = Agent(
        name="Assistant", 
        instructions="간결하게 답변하세요."
    )
    
    # 첫 번째 턴
    print("=== 첫 번째 턴 ===")
    result = await Runner.run(agent, "금문교는 어느 도시에 있나요?")
    print(f"사용자: 금문교는 어느 도시에 있나요?")
    print(f"에이전트: {result.final_output}")
    
    # 두 번째 턴 - 대화 컨텍스트 유지
    print("\n=== 두 번째 턴 (메모리 유지) ===")
    conversation_history = result.to_input_list()
    conversation_history.append({"role": "user", "content": "그 도시는 어느 주에 있나요?"})
    
    result = await Runner.run(agent, conversation_history)
    print(f"사용자: 그 도시는 어느 주에 있나요?")
    print(f"에이전트: {result.final_output}")
    
    return result

# 비동기 함수 실행
await manual_memory_demo()

=== 첫 번째 턴 ===
사용자: 금문교는 어느 도시에 있나요?
에이전트: 샌프란시스코(San Francisco)에 있습니다.

=== 두 번째 턴 (메모리 유지) ===
사용자: 그 도시는 어느 주에 있나요?
에이전트: 캘리포니아주에 있습니다.


RunResult(input=[{'content': '금문교는 어느 도시에 있나요?', 'role': 'user'}, {'id': 'msg_0e5f638cf47398ab00693580e59f58819398a12f722b1bd91e', 'content': [{'annotations': [], 'text': '샌프란시스코(San Francisco)에 있습니다.', 'type': 'output_text', 'logprobs': []}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'role': 'user', 'content': '그 도시는 어느 주에 있나요?'}], new_items=[MessageOutputItem(agent=Agent(name='Assistant', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='간결하게 답변하세요.', prompt=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, prompt_cache_retention=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], 

### 대화 기록 구조 분석

`to_input_list()`가 반환하는 대화 기록의 구조를 살펴본다. 각 메시지는 `role`(역할)과 `content`(내용) 필드를 가진 딕셔너리 형태이다. `role`은 'user', 'assistant', 'system' 중 하나이며, 이 구조를 이해하면 대화 기록을 자유롭게 조작할 수 있다.

In [8]:
async def analyze_conversation_history():
    """대화 기록 구조를 분석하는 함수이다."""
    
    agent = Agent(
        name="Assistant", 
        instructions="간결하게 답변하세요."
    )
    
    # 대화 실행
    result = await Runner.run(agent, "파이썬이란 무엇인가요?")
    
    # 대화 기록 추출
    history = result.to_input_list()
    
    print("=== 대화 기록 구조 ===")
    for i, item in enumerate(history):
        print(f"\n항목 {i+1}:")
        print(f"  타입: {type(item)}")
        if isinstance(item, dict):
            print(f"  역할(role): {item.get('role', 'N/A')}")
            content = item.get('content', 'N/A')
            if len(str(content)) > 100:
                print(f"  내용(content): {str(content)[:100]}...")
            else:
                print(f"  내용(content): {content}")
        else:
            print(f"  값: {item}")
    
    return history

history = await analyze_conversation_history()

=== 대화 기록 구조 ===

항목 1:
  타입: <class 'dict'>
  역할(role): user
  내용(content): 파이썬이란 무엇인가요?

항목 2:
  타입: <class 'dict'>
  역할(role): assistant
  내용(content): [{'annotations': [], 'text': '파이썬은 쉽고 간결한 문법을 가진 프로그래밍 언어로, 웹 개발, 데이터 분석, 인공지능 등 다양한 분야에서 사용됩니다.', '...


---

## 4. 컨텍스트 메모리 - 애플리케이션 상태 관리

이 챕터에서는 `RunContextWrapper`를 사용하여 애플리케이션 컨텍스트를 도구에 전달하는 방법을 다룬다. 대화 메모리와는 별개로 사용자 프로필, 설정, 세션 데이터 등 애플리케이션 상태를 관리할 수 있다.

### 학습 내용

- `RunContextWrapper`를 사용한 애플리케이션 컨텍스트 전달
- dataclass를 사용한 타입이 지정된 컨텍스트 객체 생성
- 사용자 설정 및 세션 데이터에 접근하는 도구 구축
- 대화 메모리와 애플리케이션 컨텍스트의 분리

### 컨텍스트 객체 정의

컨텍스트 객체는 `dataclass`를 사용하여 정의한다. 이 객체에는 사용자 ID, 사용자명, 설정 등 에이전트가 작업 수행 시 참조해야 하는 정보를 담는다. 컨텍스트는 도구 함수에서 `RunContextWrapper`를 통해 접근할 수 있으며, 이를 통해 개인화된 응답을 생성할 수 있다.

In [9]:
import os
import asyncio
from dataclasses import dataclass
from dotenv import load_dotenv
from agents import Agent, RunContextWrapper, Runner, function_tool

In [10]:
# 사용자 컨텍스트 정의
@dataclass
class UserContext:
    """사용자 정보를 담는 컨텍스트 클래스이다."""
    user_id: str
    username: str
    preferences: dict

print("UserContext 클래스가 정의되었다.")
print("필드: user_id, username, preferences")

UserContext 클래스가 정의되었다.
필드: user_id, username, preferences


### 컨텍스트 접근 도구 정의

`@function_tool` 데코레이터를 사용하여 컨텍스트에 접근하는 도구를 정의한다. 도구 함수의 첫 번째 파라미터로 `RunContextWrapper[UserContext]`를 받으면, `wrapper.context`를 통해 컨텍스트 객체에 접근할 수 있다. 이 도구는 에이전트가 사용자의 설정 정보를 조회할 때 호출된다.

In [11]:
# 컨텍스트 접근 도구 정의
@function_tool
async def get_user_preference(wrapper: RunContextWrapper[UserContext]) -> str:
    """사용자의 설정 정보를 조회하는 도구이다."""
    user = wrapper.context
    theme = user.preferences.get('theme', 'default')
    language = user.preferences.get('language', 'ko')
    return f"사용자 {user.username}의 설정: 테마={theme}, 언어={language}"

print("get_user_preference 도구가 정의되었다.")

get_user_preference 도구가 정의되었다.


### 컨텍스트 기반 에이전트 실행

제네릭 타입 `Agent[UserContext]`를 사용하여 에이전트를 생성하고, `Runner.run()`의 `context` 파라미터에 컨텍스트 객체를 전달한다. 에이전트는 필요할 때 도구를 호출하여 컨텍스트 정보에 접근할 수 있다. 컨텍스트는 도구에서만 사용 가능하며, 대화 메모리는 여전히 별도로 관리해야 한다.

In [12]:
async def context_memory_demo():
    """컨텍스트 메모리를 시연하는 함수이다."""
    
    # 사용자 데이터로 컨텍스트 생성
    user_context = UserContext(
        user_id="123",
        username="Alice", 
        preferences={"theme": "dark", "language": "ko"}
    )
    
    # 컨텍스트 타입이 지정된 에이전트 생성
    agent = Agent[UserContext](
        name="PersonalAssistant",
        instructions="당신은 도움이 되는 개인 어시스턴트입니다. 사용자 정보가 필요하면 도구를 사용하세요.",
        tools=[get_user_preference]
    )
    
    # 컨텍스트와 함께 에이전트 실행
    print("=== 컨텍스트 메모리 데모 ===")
    result = await Runner.run(
        agent, 
        "제 이름과 테마 설정이 무엇인가요?", 
        context=user_context
    )
    
    print(f"사용자: 제 이름과 테마 설정이 무엇인가요?")
    print(f"에이전트: {result.final_output}")
    
    return result

await context_memory_demo()

=== 컨텍스트 메모리 데모 ===
사용자: 제 이름과 테마 설정이 무엇인가요?
에이전트: 회원님의 이름은 Alice이고, 현재 설정된 테마는 다크 모드입니다.


RunResult(input='제 이름과 테마 설정이 무엇인가요?', new_items=[ToolCallItem(agent=Agent(name='PersonalAssistant', handoff_description=None, tools=[FunctionTool(name='get_user_preference', description='사용자의 설정 정보를 조회하는 도구이다.', params_json_schema={'properties': {}, 'title': 'get_user_preference_args', 'type': 'object', 'additionalProperties': False, 'required': []}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10c98ba60>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)], mcp_servers=[], mcp_config={}, instructions='당신은 도움이 되는 개인 어시스턴트입니다. 사용자 정보가 필요하면 도구를 사용하세요.', prompt=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, prompt_cache_retention=None, include_usage=None, response

---

## 5. 고급 컨텍스트 메모리 - 장기 기억 시스템

이 챕터에서는 컨텍스트 객체를 활용하여 커스텀 장기 기억 시스템을 구축하는 방법을 다룬다. `remember_fact`와 `recall_facts` 도구를 만들어 에이전트가 정보를 저장하고 불러올 수 있게 한다.

### 학습 내용

- 컨텍스트 객체를 사용한 커스텀 메모리 시스템 구축
- 장기 기억을 위한 `remember_fact` 및 `recall_facts` 도구 생성
- 여러 대화 세션에 걸친 영속 상태 유지
- 정보를 학습하고 유지하는 메모리 인식 에이전트 구현

### 메모리 컨텍스트 및 도구 정의

메모리 컨텍스트는 `facts` 리스트를 포함하는 dataclass로 정의한다. `remember_fact` 도구는 새로운 정보를 리스트에 추가하고, `recall_facts` 도구는 저장된 모든 정보를 반환한다. 이 패턴을 사용하면 에이전트가 사용자로부터 받은 정보를 명시적으로 저장하고 나중에 참조할 수 있다.

In [13]:
import os
import asyncio
from dotenv import load_dotenv
from dataclasses import dataclass, field
from typing import List
from agents import Agent, Runner, RunContextWrapper, function_tool

In [14]:
# 메모리 컨텍스트 정의
@dataclass
class MemoryCtx:
    """간단한 키-값 메모리를 담는 컨텍스트이다."""
    facts: List[str] = field(default_factory=list)

# 메모리 저장 도구
@function_tool
async def remember_fact(wrapper: RunContextWrapper[MemoryCtx], fact: str) -> str:
    """사용자가 제공한 정보를 장기 메모리에 저장한다."""
    wrapper.context.facts.append(fact)
    return "알겠습니다. 기억하겠습니다."

# 메모리 조회 도구
@function_tool
async def recall_facts(wrapper: RunContextWrapper[MemoryCtx]) -> str:
    """어시스턴트가 지금까지 알고 있는 모든 정보를 반환한다."""
    if not wrapper.context.facts:
        return "메모리가 비어 있습니다."
    return "\n".join(wrapper.context.facts)

print("메모리 컨텍스트와 도구가 정의되었다.")
print("- remember_fact: 정보 저장")
print("- recall_facts: 정보 조회")

메모리 컨텍스트와 도구가 정의되었다.
- remember_fact: 정보 저장
- recall_facts: 정보 조회


### 메모리 인식 에이전트 생성

에이전트의 지시사항에 메모리 도구 사용 방법을 명시한다. 사용자가 전기적 정보나 선호도를 알려주면 `remember_fact`를 호출하여 저장하고, 확실하지 않을 때는 `recall_facts`를 호출하여 메모리를 확인하도록 안내한다. 이렇게 하면 에이전트가 자율적으로 메모리를 관리할 수 있다.

In [15]:
# 메모리 인식 에이전트 생성
memo_agent = Agent[MemoryCtx](
    name="MemoAssistant",
    model=MODEL,
    instructions=(
        "당신은 시간이 지나도 정보를 *기억*할 수 있는 도움이 되는 어시스턴트입니다.\n"
        "- 사용자가 전기적 정보나 선호도를 알려주면, 잊지 않도록 `remember_fact`를 "
        "관련 문장과 함께 호출하세요.\n"
        "- 확실하지 않을 때는 `recall_facts`를 호출하여 메모리를 새로고침하세요."
    ),
    tools=[remember_fact, recall_facts],
)

print(f"메모리 인식 에이전트 '{memo_agent.name}'가 생성되었다.")

메모리 인식 에이전트 'MemoAssistant'가 생성되었다.


### 장기 메모리 시스템 테스트

단일 `MemoryCtx` 인스턴스를 생성하고 여러 대화 턴에 걸쳐 재사용한다. 첫 번째 상호작용에서 사용자가 정보를 알려주면 에이전트가 이를 메모리에 저장하고, 두 번째 상호작용에서 해당 정보를 물어보면 저장된 정보를 기반으로 응답한다. 컨텍스트 객체가 동일하게 유지되므로 정보가 보존된다.

In [16]:
async def advanced_memory_demo():
    """고급 메모리 시스템을 시연하는 함수이다."""
    
    # 모든 턴에서 재사용할 단일 MemoryCtx 생성
    memory = MemoryCtx()
    
    print("=== 고급 컨텍스트 메모리 데모 ===")
    
    # 첫 번째 상호작용 - 사용자가 에이전트에게 정보를 알려준다
    print("\n[첫 번째 상호작용]")
    print("사용자: 제 생일은 6월 29일입니다.")
    
    await Runner.run(
        starting_agent=memo_agent,
        input="제 생일은 6월 29일입니다.",
        context=memory,
    )
    
    print(f"메모리 상태: {memory.facts}")
    
    # 두 번째 상호작용 - 모델이 저장된 정보를 불러와야 한다
    print("\n[두 번째 상호작용]")
    print("사용자: 제 생일이 언제죠?")
    
    result = await Runner.run(
        starting_agent=memo_agent,
        input="제 생일이 언제죠?",
        context=memory,
    )
    
    print(f"에이전트: {result.final_output}")
    
    return memory

memory_result = await advanced_memory_demo()

=== 고급 컨텍스트 메모리 데모 ===

[첫 번째 상호작용]
사용자: 제 생일은 6월 29일입니다.
메모리 상태: ['사용자의 생일은 6월 29일입니다.']

[두 번째 상호작용]
사용자: 제 생일이 언제죠?
에이전트: 당신의 생일은 6월 29일입니다.


### 추가 정보 저장 테스트

동일한 메모리 컨텍스트에 추가 정보를 저장하고 전체 메모리를 확인한다. 메모리는 누적되므로 이전에 저장한 정보와 새로 저장한 정보가 함께 유지된다.

In [17]:
async def add_more_facts(memory: MemoryCtx):
    """기존 메모리에 추가 정보를 저장한다."""
    
    print("\n=== 추가 정보 저장 ===")
    
    # 추가 정보 저장
    print("사용자: 저는 서울에 살고 있고, 좋아하는 색은 파란색입니다.")
    await Runner.run(
        starting_agent=memo_agent,
        input="저는 서울에 살고 있고, 좋아하는 색은 파란색입니다.",
        context=memory,
    )
    
    # 전체 메모리 확인
    print("\n사용자: 저에 대해 뭘 알고 있나요?")
    result = await Runner.run(
        starting_agent=memo_agent,
        input="저에 대해 뭘 알고 있나요?",
        context=memory,
    )
    
    print(f"에이전트: {result.final_output}")
    print(f"\n최종 메모리 상태: {memory.facts}")

await add_more_facts(memory_result)


=== 추가 정보 저장 ===
사용자: 저는 서울에 살고 있고, 좋아하는 색은 파란색입니다.

사용자: 저에 대해 뭘 알고 있나요?
에이전트: 저는 당신의 생일이 6월 29일이고, 서울에 살며, 좋아하는 색이 파란색이라는 것을 알고 있습니다. 더 알고 싶은 정보가 있으면 말씀해 주세요!

최종 메모리 상태: ['사용자의 생일은 6월 29일입니다.', '사용자는 서울에 살고 있다.', '사용자가 좋아하는 색은 파란색이다.']


---

## 6. 복합 메모리와 컨텍스트

이 챕터에서는 대화 메모리와 애플리케이션 컨텍스트를 결합하는 방법을 다룬다. 대화 기록(수동 메모리)과 세션 데이터(컨텍스트)를 함께 관리하여 포괄적인 상태 관리를 구현한다.

### 학습 내용

- 대화 기록(수동 메모리)과 애플리케이션 컨텍스트의 동시 사용
- 대화 상태와 함께 세션 데이터 관리
- 대화 컨텍스트와 사용자 컨텍스트를 모두 유지하는 에이전트 구축
- 프로덕션 대화형 애플리케이션을 위한 실용적인 패턴

### 애플리케이션 컨텍스트 정의

프로덕션 환경에서는 사용자 ID, 세션 ID 등의 애플리케이션 상태와 대화 기록을 함께 관리해야 한다. 애플리케이션 컨텍스트는 인증, 로깅, 분석 등에 필요한 메타데이터를 담고, 대화 기록은 멀티턴 대화의 맥락을 유지한다.

In [18]:
import os
import asyncio
from dataclasses import dataclass
from dotenv import load_dotenv
from agents import Agent, Runner

In [19]:
# 애플리케이션 컨텍스트 정의
@dataclass
class AppContext:
    """애플리케이션 세션 정보를 담는 컨텍스트이다."""
    user_id: str
    session_id: str

print("AppContext 클래스가 정의되었다.")
print("필드: user_id, session_id")

AppContext 클래스가 정의되었다.
필드: user_id, session_id


### 복합 메모리 패턴 구현

대화 메모리(`to_input_list()`)와 애플리케이션 컨텍스트(`context` 파라미터)를 함께 사용한다. 첫 번째 호출 후 대화 기록을 추출하고, 새 메시지를 추가한 후 동일한 컨텍스트와 함께 다음 호출에 전달한다. 이렇게 하면 에이전트는 이전 대화 내용을 기억하면서 애플리케이션 상태에도 접근할 수 있다.

In [20]:
async def chat_with_combined_memory():
    """복합 메모리 패턴을 시연하는 함수이다."""
    
    # 애플리케이션 컨텍스트 생성
    context = AppContext(user_id="123", session_id="session_456")
    
    # 컨텍스트 타입이 지정된 에이전트 생성
    agent = Agent[AppContext](
        name="Assistant", 
        instructions="도움이 되는 어시스턴트입니다. 대화의 맥락을 유지하세요."
    )
    
    print("=== 복합 메모리 데모 ===")
    print(f"세션 정보: user_id={context.user_id}, session_id={context.session_id}")
    
    # 첫 번째 대화
    print("\n[첫 번째 턴]")
    print("사용자: 안녕하세요, 저는 웹사이트를 만들고 있어요.")
    
    result = await Runner.run(
        agent, 
        "안녕하세요, 저는 웹사이트를 만들고 있어요.", 
        context=context
    )
    print(f"에이전트: {result.final_output}")
    
    # 메모리와 함께 대화 계속
    print("\n[두 번째 턴 - 대화 기록 유지]")
    print("사용자: 어떤 기술을 사용해야 할까요?")
    
    history = result.to_input_list()
    history.append({"role": "user", "content": "어떤 기술을 사용해야 할까요?"})
    
    result = await Runner.run(agent, history, context=context)
    print(f"에이전트: {result.final_output}")
    
    # 세 번째 턴
    print("\n[세 번째 턴 - 대화 기록 계속 유지]")
    print("사용자: 그 중에서 가장 쉬운 것은 무엇인가요?")
    
    history = result.to_input_list()
    history.append({"role": "user", "content": "그 중에서 가장 쉬운 것은 무엇인가요?"})
    
    result = await Runner.run(agent, history, context=context)
    print(f"에이전트: {result.final_output}")
    
    return result

final_result = await chat_with_combined_memory()

=== 복합 메모리 데모 ===
세션 정보: user_id=123, session_id=session_456

[첫 번째 턴]
사용자: 안녕하세요, 저는 웹사이트를 만들고 있어요.
에이전트: 안녕하세요! 웹사이트 제작 중이시군요. 멋집니다!  
어떤 종류의 웹사이트를 만들고 계신가요?  
특별히 궁금한 점, 예를 들어 HTML, CSS, JavaScript, 프레임워크(React, Vue 등), 배포 방법 등 궁금하신 내용이 있으시면 언제든 말씀해 주세요.  
어떤 부분을 도와드릴까요?

[두 번째 턴 - 대화 기록 유지]
사용자: 어떤 기술을 사용해야 할까요?
에이전트: 좋은 질문입니다!  
웹사이트의 목적과 규모, 그리고 본인의 개발 경험에 따라 사용할 기술이 달라질 수 있어요.  
아래 질문에 답해주시면 더 맞춤으로 추천드릴 수 있습니다.

1. 웹사이트의 목적이 무엇인가요? (예: 블로그, 포트폴리오, 쇼핑몰, 커뮤니티 등)
2. 동적인 기능(로그인, 게시글 쓰기, 댓글 등)이 필요한가요, 아니면 주로 정보 제공용인가요?
3. 개발 경험이나 선호하는 언어/프레임워크가 있으신가요? (예: HTML/CSS만 가능, JavaScript 사용 가능 등)
4. 혼자 만드시나요, 아니면 팀 작업인가요?
5. 배포(호스팅)까지 계획이 있으신가요? (예: 어디에 올릴 예정인지 등)

대략적인 가이드라인은 아래와 같습니다.

---

### 1. **정적 웹사이트(정보 제공용, 동적 기능 거의 없음)**
- **필수:** HTML, CSS
- **선택:** JavaScript (필요하면)
- **빌더 사용:** [Wix](https://www.wix.com), [WordPress.com](https://www.wordpress.com), [Notion + Super.so](https://super.so) 등

### 2. **동적 웹사이트(로그인, 게시판 등)**
- **프론트엔드:**  
  - React, Vue, 또는 Next.js(React 기반), Nuxt.j

### 대화 기록 분석

최종 대화 기록을 분석하여 전체 대화 흐름을 확인한다. 각 턴에서의 사용자 입력과 에이전트 응답이 순서대로 기록되어 있음을 볼 수 있다.

In [21]:
# 최종 대화 기록 분석
print("=== 최종 대화 기록 분석 ===")
final_history = final_result.to_input_list()

turn_count = 0
for item in final_history:
    if isinstance(item, dict):
        role = item.get('role', 'unknown')
        content = item.get('content', '')
        
        if role == 'user':
            turn_count += 1
            print(f"\n--- 턴 {turn_count} ---")
            print(f"[사용자] {content[:100]}{'...' if len(str(content)) > 100 else ''}")
        elif role == 'assistant':
            print(f"[에이전트] {str(content)[:100]}{'...' if len(str(content)) > 100 else ''}")

print(f"\n총 대화 턴 수: {turn_count}")

=== 최종 대화 기록 분석 ===

--- 턴 1 ---
[사용자] 안녕하세요, 저는 웹사이트를 만들고 있어요.
[에이전트] [{'annotations': [], 'text': '안녕하세요! 웹사이트 제작 중이시군요. 멋집니다!  \n어떤 종류의 웹사이트를 만들고 계신가요?  \n특별히 궁금한 점, 예를...

--- 턴 2 ---
[사용자] 어떤 기술을 사용해야 할까요?
[에이전트] [{'annotations': [], 'text': '좋은 질문입니다!  \n웹사이트의 목적과 규모, 그리고 본인의 개발 경험에 따라 사용할 기술이 달라질 수 있어요.  \n아래 ...

--- 턴 3 ---
[사용자] 그 중에서 가장 쉬운 것은 무엇인가요?
[에이전트] [{'annotations': [], 'text': '아주 좋은 질문입니다!  \n웹사이트를 처음 만들거나 개발 경험이 많지 않다면, 아래 순서대로 점점 난이도가 올라간다고 생각하...

총 대화 턴 수: 3


### 메모리 패턴 비교 요약

지금까지 학습한 메모리 패턴을 정리한다. 각 패턴은 서로 다른 사용 사례에 적합하며, 필요에 따라 조합하여 사용할 수 있다.

=== 메모리 패턴 비교 요약 ===

1. 무상태 (Chapter 8)
   - 각 호출이 독립적
   - 이전 대화를 기억하지 않음
   - 사용 사례: 단일 질의응답, 상태 불필요한 작업

2. 영속 메모리 - run_demo_loop (Chapter 9)
   - 자동 대화 기록 관리
   - 터미널 기반 대화형 세션
   - 사용 사례: 프로토타이핑, 간단한 챗봇

3. 수동 메모리 - to_input_list() (Chapter 10)
   - 대화 기록에 대한 세밀한 제어
   - 기억할 내용 선택 가능
   - 사용 사례: 커스텀 대화 흐름, 선택적 메모리

4. 컨텍스트 메모리 - RunContextWrapper (Chapter 11)
   - 애플리케이션 상태를 도구에 전달
   - 대화 메모리와 분리된 사용자 데이터
   - 사용 사례: 개인화, 사용자 설정 접근

5. 고급 컨텍스트 메모리 (Chapter 12)
   - 커스텀 장기 기억 시스템
   - 도구를 통한 명시적 저장/조회
   - 사용 사례: 장기 사용자 정보 관리, 학습하는 에이전트

6. 복합 메모리 (Chapter 13)
   - 대화 기록 + 애플리케이션 컨텍스트
   - 포괄적인 상태 관리
   - 사용 사례: 프로덕션 대화형 애플리케이션

---

## 마무리

이 튜토리얼에서는 OpenAI Agents SDK의 다양한 메모리 관리 패턴을 다루었다. 다음과 같은 내용을 학습하였다:

1. **무상태 동작**: 에이전트의 기본 동작과 메모리 관리의 필요성
2. **영속 메모리**: `run_demo_loop`를 사용한 자동 대화 기록 관리
3. **수동 메모리**: `to_input_list()`를 사용한 세밀한 대화 컨텍스트 제어
4. **컨텍스트 메모리**: `RunContextWrapper`를 사용한 애플리케이션 상태 전달
5. **고급 컨텍스트 메모리**: 커스텀 장기 기억 시스템 구축
6. **복합 메모리**: 대화 기록과 애플리케이션 컨텍스트의 결합

이러한 패턴을 적절히 조합하면 대화의 맥락을 유지하고, 사용자 정보를 기억하며, 개인화된 경험을 제공하는 정교한 에이전트 시스템을 구축할 수 있다.